In [138]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f

In [229]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
#BATCH=30
#BATCH=100 
BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 800


In [159]:
# em numero de frases
#BATCH=30
#BATCH=100 
BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
#print(dicSentences_new_test[0])
print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))


sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
print('verificando dados:')
for key, value in dic_predictions.items():
    print('key:',key)
    print(dic_predictions[key])
    if key>2:
        break
        
print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
#print(dic_predictions[8])
#print(dicSentences_new_test[8][1])
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


BATCH: 800
Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Nega', 0, 422], ['sincope', 1, 427], ['.', 2, 434]], [['sincope', [1], 'Problema']]]
numero de sentencas no total: 506
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
[[['Abd', 0], ['globoso', 1], [',', 2], ['flacido', 3], [',', 4], ['indolor', 5], ['a', 6], ['palpacao', 7], [',', 8], ['sem', 9], ['VCM', 10], ['.', 11]], [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]
verificando dados:
key: 0
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
key: 1
[[['Em', 0], ['acompanhamento', 1], ['no', 2], ['ambualtorio', 3], ['há', 4], ['5', 5], ['anos', 6], ['por', 7], ['FA', 8], [',', 9], ['uso', 10], ['de', 11], ['marevan', 12], ['5mg', 13], ['1', 14], ['x', 15], ['ao', 16], ['dia', 17], ['.', 18]], [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]

In [212]:
def getCombinacaoEntidades(dic_predictions, filtro_postagger, dicPosTagger, lista_postaggers_entidades):
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesAll = list()
    combinacaoEntidades = list()
    pulando_termos_postagger = list()
    if filtro_postagger:
        print('Com filtro-postagger')
    else:
        print('Sem filtro-postagger')
    for key, value in dic_predictions.items():
        num=num+1
        combinacaoEntidades = list()
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            frase.insert(inicio, '<e1>')
            frase.insert(fim+2, '</e1>')
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = f.replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio+1:fim+2]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = f.replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if (texto_entidade_comparar == texto_frase_comparar):
                combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])

        for entidade in entidades:
                indices = entidade[1]
                print('indices:', indices)
                if indices in erros_entidade:
                    continue
                inicio=indices[0]
                fim=indices[-1]
                print('inicio:', inicio)
                print('fim:', fim)
                # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
                for indice in indices:
                    print('indice:', indice)
                    for i in range(indice, fim+1):
                        # ver se nao tem antes
                        print('i:', i)
                        frase = so_tokens.copy()
                        #termo = frase[indice:i+2]
                        termo = frase[indice:i+1]
                        frase.insert(indice, '<e1>')
                        frase.insert(i+2, '</e1>')
                        frase_string=' '.join(frase).strip()
                        print('---frase_string:---', frase_string)
                        print('termo', termo)
                        devePular = 0
                        if '. </e1>' in frase_string or ', </e1>' in frase_string  or '; </e1>' in frase_string or '- </e1>' in frase_string  or ': </e1>' in frase_string  or '= </e1>' in frase_string  or '/ </e1>' in frase_string  or '( </e1>' in frase_string  or ') </e1>' in frase_string  or '[ </e1>' in frase_string  or '] </e1>' in frase_string  or ': </e1>' in frase_string or 'and </e1>' in frase_string or 'or </e1>' in frase_string:
                            devePular=1
                            print('devePular1')
                        if '<e1> .' in frase_string or '<e1> ,' in frase_string  or '<e1> ;' in frase_string or '<e1> -' in frase_string  or '<e1> :' in frase_string  or '<e1> =' in frase_string  or '<e1> /' in frase_string  or '<e1> (' in frase_string  or '<e1> )' in frase_string  or '<e1> [' in frase_string  or '<e1> ]' in frase_string  or '<e1> :' in frase_string  or '<e1> and' in frase_string  or '<e1> or' in frase_string:
                            devePular=1
                            print('devePular2')
                        if re.search("<e1> [0-9]* </e1>", frase_string):
                            devePular=1
                            print('devePular3')
                        if filtro_postagger==True:
                            pos_tagger_termo = f.tipoPostaggerTokens(termo, dicPosTagger)
                            print('pos_tagger_termo:', pos_tagger_termo)
                            if pos_tagger_termo not in lista_postaggers_entidades:
                                pulando_termos_postagger.append([termo, pos_tagger_termo])
                                devePular=1
                                print('devePular4:', termo)
                
                        tem_frase = 0
                        for frase in combinacaoEntidades:
                            if frase[0] == frase_string:
                                tem_frase=''
                                break
                        if tem_frase==0 and devePular==0:
                            combinacaoEntidades.append([frase_string, 'O'])
                        else:
                            print('deve pular:', frase_string)
        # shuffle no combinacaoEntidades
        if len(combinacaoEntidades)>0:
            combinacaoEntidadesAll.append([' '.join(so_tokens).strip(), combinacaoEntidades])
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
            combinacaoEntidadesAll.append([])
        combinacaoEntidades = list()
        if (num % 1000) ==0:
            print('key:', key)

    print('len(combinacaoEntidadesAll:)',len(combinacaoEntidadesAll))
    print('len(pulando_termos_postagger):', len(pulando_termos_postagger))
    
    return combinacaoEntidadesAll, pulando_termos_postagger

dic_predictions2={}
dic_predictions2[0] = dic_predictions[14].copy()
print(dic_predictions2)
a, b = getCombinacaoEntidades(dic_predictions2, True, dicPosTagger, lista_postaggers_entidades)

print('\na:', a)
print('\nb:',b)

{0: [[['aumento', 0], ['moderado', 1], ['de', 2], ['átrio', 3], ['esquerdo', 4], ['.', 5]], [['aumento moderado de átrio esquerdo', [0, 1, 2, 3, 4], 'Problema']]]}
Com filtro-postagger
indices: [0, 1, 2, 3, 4]
inicio: 0
fim: 4
indice: 0
i: 0
---frase_string:--- <e1> aumento </e1> moderado de átrio esquerdo .
termo ['aumento']
pos_tagger_termo: -V
i: 1
---frase_string:--- <e1> aumento moderado </e1> de átrio esquerdo .
termo ['aumento', 'moderado']
pos_tagger_termo: -V-N
i: 2
---frase_string:--- <e1> aumento moderado de </e1> átrio esquerdo .
termo ['aumento', 'moderado', 'de']
pos_tagger_termo: -V-N-PREP
devePular4: ['aumento', 'moderado', 'de']
deve pular: <e1> aumento moderado de </e1> átrio esquerdo .
i: 3
---frase_string:--- <e1> aumento moderado de átrio </e1> esquerdo .
termo ['aumento', 'moderado', 'de', 'átrio']
pos_tagger_termo: -V-N-PREP-N
devePular4: ['aumento', 'moderado', 'de', 'átrio']
deve pular: <e1> aumento moderado de átrio </e1> esquerdo .
i: 4
---frase_string:--- <e

### Agora, modelo de SpanClassification

In [216]:
print('-----Avaliando só com modelo de Spam:-----')
model = f.getModel()
dic_predictions =  f.load_obj('dic_predictions_results_ner_'+str(BATCH)) # TODO
dic_sentencesTrainDev = f.load_obj('dic_sentencesTrainDev')
dicPosTagger, _ = f.getDicPosTagger(dic_sentencesTrainDev)
lista_postaggers_entidades = f.getListaPostaggerEntidades(dic_predictions, dicPosTagger)
# para gerar arquivo de predicoes (com as tags <e1>)
print('Primeiro, com filtro postagger:')
combinacaoEntidadesAll, pullando_termos_pos_tagger = f.getCombinacaoEntidades(dic_predictions, True, dicPosTagger, lista_postaggers_entidades)
f.save_obj('combinacaoEntidadesAll_comPostagger_'+str(BATCH), combinacaoEntidadesAll)
f.gravaArquivoPredict(combinacaoEntidadesAll, tipo='io', postagger=True)
print('Alguns termos pulados:')
print(pullando_termos_pos_tagger[:5])
# chamar predict
print('Chamando predict')
pred_region_labels = f.predictSpan(model)
print('len(pred_region_labels):', len(pred_region_labels))
print(region_true_list[:10])
print(pred_region_labels[:10])
#combinacaoEntidadesAll = load_obj(combinacaoEntidadesAll_'+str(BATCH))

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)
print('vai chamar getDicPredictionsSpan')
dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)
f.save_obj('dic_predictions_span_com_postagger_'+str(BATCH), dic_predictions_span)
print('vai chamar AvalFinal')
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)

-----Avaliando só com modelo de Spam:-----
loading model from C:\Users\lisat\OneDrive\jupyter notebook\span-model\model-exp3
num_labels: 5
Primeiro, com filtro postagger:
Com filtro-postagger
len(combinacaoEntidadesAll:) 506
len(pulando_termos_postagger): 1107
numTotalEntidades: 2240
numFrases: 411
gravando em: data/span.compostagger.predict
Alguns termos pulados:
[[['25'], '-NUM'], [['hipertrofia', 'concentrica', 'de'], '-N-N-PREP'], [['concentrica', 'de'], '-N-PREP'], [['de', 'grau', 'discreto'], '-PREP-N-ADJ'], [['aumento', 'moderado', 'de'], '-V-N-PREP']]
Chamando predict
['O', 'Problema', 'Tratamento', 'Teste', 'Anatomia']
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
LOOKING AT ./data\span.predict
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
*** Example ***
tokens: ['[CLS]', 'Em', 'ac', '##om', '##pan', '##ham', '##ento', 'n

In [235]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)


print('-----TESTE, deletar-----')
print('Chamando predict')
pred_region_labels = f.predictSpan2(model, 512)
print('len(pred_region_labels):', len(pred_region_labels))
#print(region_true_list[:50])
print(pred_region_labels[:50])
#combinacaoEntidadesAll = load_obj(combinacaoEntidadesAll_'+str(BATCH))

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
#combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)
#print('vai chamar getDicPredictionsSpan')
#dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)
#f.save_obj('dic_predictions_span_com_postagger_'+str(BATCH), dic_predictions_span)
#print('vai chamar AvalFinal')
#region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)

-----TESTE, deletar-----
Chamando predict
['O', 'Problema', 'Tratamento', 'Teste', 'Anatomia']
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
LOOKING AT ./data\span.predict
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
*** Example ***
tokens: ['[CLS]', 'SO', '##L', '##IC', '##IT', '##O', 'EX', '##AM', '##ES', 'LA', '##B', '##OR', '##AT', '##OR', '##IA', '##IS', 'E', 'CA', '##TE', '##TER', '##IS', '##M', '##O', 'DI', '##AG', '##NO', '##ST', '##IC', '##O', 'PA', '##RA', 'V', '##ER', 'OS', '##ST', '##EN', '##TS', 'IM', '##P', '##LA', '##NT', '##AD', '##OS', '.', '[SEP]']
lista_label: [0, 0, 2, 3, 0, 3]
indices_e1_sentenca: [[6, 8], [9, 15], [33, 36], [17, 28], [17, 22], [6, 15]]
*** Example ***
tokens: ['[CLS]', 'HM', '##A', ':', 'paciente', 're', '##lata', 'can', '##sa', '##ço', ',', 'dor', 'pre', '##cor', '##dia', '##l', 'quando', 't

In [222]:
print('agora, com  filtro pos tagger')

#dic_predictions_span = f.load_obj('dic_predictions_span_com_postagger_'+str(BATCH))
dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_span)
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

agora, com  filtro pos tagger
Não é aninhada, incluindo: ['glicazida', [22], 'Problema']
Não é aninhada, incluindo: ['comprometimento difuso do VE', [33, 34, 35, 36], 'Anatomia']
Não é aninhada, incluindo: ['VE', [36], 'Anatomia']
Não é aninhada, incluindo: ['VE grau', [36, 37], 'Anatomia']
Não é aninhada, incluindo: ['VE grau moderado', [36, 37, 38], 'Anatomia']
Não é aninhada, incluindo: ['grau', [37], 'Anatomia']
Não é aninhada, incluindo: ['insuf Mi', [40, 41], 'Teste']
Não é aninhada, incluindo: ['discreta', [42], 'Teste']
Não é aninhada, incluindo: ['insuf Tri', [44, 45], 'Anatomia']
Não é aninhada, incluindo: ['Tri discreta', [45, 46], 'Teste']
Não é aninhada, incluindo: ['discreta', [46], 'Teste']
Não é aninhada, incluindo: ['em MMII', [4, 5], 'Anatomia']
Não é aninhada, incluindo: ['MMII', [5], 'Anatomia']
Não é aninhada, incluindo: ['toracica', [2], 'Anatomia']
Não é aninhada, incluindo: ['total', [22], 'Problema']
Não é aninhada, incluindo: ['tróficas', [27], 'Teste']
Não é 

In [188]:
dic_predictions[14]

[[['aumento', 0],
  ['moderado', 1],
  ['de', 2],
  ['átrio', 3],
  ['esquerdo', 4],
  ['.', 5]],
 [['aumento moderado de átrio esquerdo', [0, 1, 2, 3, 4], 'Problema']]]

In [220]:
pullando_termos_pos_tagger[:50]

[[['25'], '-NUM'],
 [['hipertrofia', 'concentrica', 'de'], '-N-N-PREP'],
 [['concentrica', 'de'], '-N-PREP'],
 [['de', 'grau', 'discreto'], '-PREP-N-ADJ'],
 [['aumento', 'moderado', 'de'], '-V-N-PREP'],
 [['aumento', 'moderado', 'de', 'átrio'], '-V-N-PREP-N'],
 [['moderado', 'de'], '-N-PREP'],
 [['de', 'átrio', 'esquerdo'], '-PREP-N-ADJ'],
 [['alterações', 'nos'], '-N-ART'],
 [['nos', 'MMII'], '-ART-N'],
 [['dose', 'da'], '-N-ART'],
 [['dose', 'da', 'sinvastatina', 'para'], '-N-ART-N-PREP'],
 [['da', 'sinvastatina'], '-ART-N'],
 [['da', 'sinvastatina', 'para'], '-ART-N-PREP'],
 [['da', 'sinvastatina', 'para', '40mg'], '-ART-N-PREP-N'],
 [['sinvastatina', 'para'], '-N-PREP'],
 [['comprometimento', 'difuso', 'do'], '-N-N-ART'],
 [['comprometimento', 'difuso', 'do', 'VE', 'grau'], '-N-N-ART-N-N'],
 [['difuso', 'do'], '-N-ART'],
 [['do', 'VE'], '-ART-N'],
 [['do', 'VE', 'grau'], '-ART-N-N'],
 [['do', 'VE', 'grau', 'moderado'], '-ART-N-N-N'],
 [['pulmonar', 'discreta'], '-ADJ-ADJ'],
 [['fra

In [219]:
combinacaoEntidadesAll[14]

['aumento moderado de átrio esquerdo .',
 [['<e1> aumento moderado de átrio esquerdo </e1> .', 'Problema'],
  ['<e1> aumento </e1> moderado de átrio esquerdo .', 'O'],
  ['<e1> aumento moderado </e1> de átrio esquerdo .', 'O'],
  ['aumento <e1> moderado </e1> de átrio esquerdo .', 'O'],
  ['aumento <e1> moderado de átrio </e1> esquerdo .', 'O'],
  ['aumento <e1> moderado de átrio esquerdo </e1> .', 'O'],
  ['aumento moderado <e1> de </e1> átrio esquerdo .', 'O'],
  ['aumento moderado <e1> de átrio </e1> esquerdo .', 'O'],
  ['aumento moderado de <e1> átrio </e1> esquerdo .', 'O'],
  ['aumento moderado de <e1> átrio esquerdo </e1> .', 'O'],
  ['aumento moderado de átrio <e1> esquerdo </e1> .', 'O']]]

In [165]:
len(dicSentences_new_test)

506

In [168]:
# para gerar arquivo de predicoes (com as tags <e1>)
print('Agora, sem filtro postagger:')
combinacaoEntidadesAll, pullando_termos_pos_tagger = f.getCombinacaoEntidades(dic_predictions, False, '', '')
f.save_obj('combinacaoEntidadesAll_comPostagger_'+str(BATCH), combinacaoEntidadesAll)
f.gravaArquivoPredict(combinacaoEntidadesAll, tipo='io', postagger=False)
# chamar predict
print('Chamando predict')
pred_region_labels = f.predictSpan(model)
print('len(pred_region_labels):', len(pred_region_labels))
print(region_true_list[:10])
print(pred_region_labels[:50])
#combinacaoEntidadesAll = load_obj(combinacaoEntidadesAll_'+str(BATCH))

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)

dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)
f.save_obj('dic_predictions_span_sem_postagger_'+str(BATCH), dic_predictions_span)

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)

Agora, sem filtro postagger:
Sem filtro-postagger
len(combinacaoEntidadesAll:) 506
len(pulando_termos_postagger): 0
numTotalEntidades: 2896
numFrases: 411
gravando em: data/span.sempostagger.predict
Chamando predict
['O', 'Problema', 'Tratamento', 'Teste', 'Anatomia']
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
LOOKING AT ./data\span.predict
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
os.getcwd(): C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\avaliacao
*** Example ***
tokens: ['[CLS]', 'Em', 'ac', '##om', '##pan', '##ham', '##ento', 'no', 'amb', '##ual', '##torio', 'há', '5', 'anos', 'por', 'FA', ',', 'uso', 'de', 'mare', '##van', '5', '##m', '##g', '1', 'x', 'ao', 'dia', '.', '[SEP]']
lista_label: [1, 2, 0, 0]
indices_e1_sentenca: [[15, 15], [19, 23], [19, 20], [21, 23]]
*** Example ***
tokens: ['[CLS]', 'Como', '##rb', '##idades', ':', 'DM', 'há', '10', 'anos', 'em', 'uso', 'de', 'met', '##for

In [144]:
dicSentences_new_test

{0: [[['Lucas', 0, 43],
   [',', 1, 48],
   ['74', 2, 50],
   ['anos', 3, 53],
   ['.', 4, 57]],
  []],
 1: [[['Em', 0, 59],
   ['acompanhamento', 1, 62],
   ['no', 2, 77],
   ['ambualtorio', 3, 80],
   ['há', 4, 92],
   ['5', 5, 95],
   ['anos', 6, 97],
   ['por', 7, 102],
   ['FA', 8, 106],
   [',', 9, 108],
   ['uso', 10, 110],
   ['de', 11, 114],
   ['marevan', 12, 117],
   ['5mg', 13, 125],
   ['1', 14, 129],
   ['x', 15, 131],
   ['ao', 16, 133],
   ['dia', 17, 136],
   ['.', 18, 139]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0, 142],
   [':', 1, 154],
   ['DM', 2, 156],
   ['há', 3, 159],
   ['10', 4, 162],
   ['anos', 5, 165],
   ['em', 6, 170],
   ['uso', 7, 173],
   ['de', 8, 177],
   ['metformina', 9, 180],
   ['850mg', 10, 191],
   ['3', 11, 197],
   ['cp', 12, 199],
   ['/', 13, 201],
   ['dia', 14, 202],
   [',', 15, 205],
   ['acarbose', 16, 207],
   ['1', 17, 216],
   ['cp', 18, 218],
   ['/', 19, 220],
   ['dia', 20

In [145]:
dic_predictions

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-', 34],
 

In [221]:
dic_predictions_span

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['marevan 5mg', [12, 13], 'Tratamento'], ['marevan', [12], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-'

In [147]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)


<module 'model' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\model.py'>

In [148]:
a=[[0],[1,2]]
[1,2] in a

True

In [149]:
dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions

In [169]:
# juntar all_predictions
print('juntando span + ner')
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)

dic_predictions_all = {}
for key, value in dic_predictions.items():
    #dic_predictions_all[key] = value.copy()
    tokens = value[0].copy()
    listaEntidadesNer = value[1].copy()
    listaIndicesNer=list()
    for entidadeNer in listaEntidadesNer:
        listaIndicesNer.append(entidadeNer[1])
    #print('listaIndicesNer:', listaIndicesNer)
    listaEntidadesSpan = dic_predictions_span[key][1]
    for entidadeSpan in listaEntidadesSpan:
        indices = entidadeSpan[1]
        if indices in listaIndicesNer:
            #print('ja tem, pulando')
            pass
        else:
            #print('incluindo:', entidadeSpan)
            listaEntidadesNer.append(entidadeSpan)
    dic_predictions_all[key] = [tokens, listaEntidadesNer]
    #if key>5:
    #    break

    

juntando span + ner


In [171]:
dic_predictions_all[13]

[[['Ecocardiograma', 0],
  ['-', 1],
  ['ventrículo', 2],
  ['esquerdo', 3],
  ['com', 4],
  ['hipertrofia', 5],
  ['concentrica', 6],
  ['de', 7],
  ['grau', 8],
  ['discreto', 9],
  ['e', 10],
  ['função', 11],
  ['sistólica', 12],
  ['preservada', 13],
  ['.', 14]],
 [['Ecocardiograma', [0], 'Teste'],
  ['ventrículo esquerdo', [2, 3], 'Anatomia'],
  ['hipertrofia concentrica de grau discreto', [5, 6, 7, 8, 9], 'Problema'],
  ['ventrículo', [2], 'Anatomia'],
  ['hipertrofia', [5], 'Problema'],
  ['hipertrofia concentrica', [5, 6], 'Problema']]]

In [170]:
len(dic_predictions_span)

506

In [172]:
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

numErro1: 141
numErro2: 930
              precision    recall  f1-score   support

    Anatomia   0.370149  0.632653  0.467043       196
           O   0.000000  0.000000  0.000000       930
    Problema   0.366584  0.869822  0.515789       338
       Teste   0.678019  0.901235  0.773852       243
  Tratamento   0.571875  0.855140  0.685393       214

    accuracy                       0.426861      1921
   macro avg   0.397325  0.651770  0.488416      1921
weighted avg   0.251741  0.426861  0.312648      1921



### Com pos-processamento

Regras:

1) primeiro, aceitar sempre do primeiro modelo (que pega as entidades de fora)
2) depois, pega os de dentro (aninhadas), apenas se o tipo for diferente do tipo da entidade de fora... se for do mesmo tipo, não pega...

In [173]:
def isEntidadeAninhada(a, b):
    inicio1=a[0]
    fim1=a[-1]
    #print(inicio1, fim1)
    inicio2=b[0]
    fim2=b[-1]
    #print(inicio2, fim2)
    if inicio1 >=inicio2 and fim1 <=fim2:
        return True
    else:
        return False
isEntidadeAninhada([0], [2, 3])

False

In [175]:
# juntar all_predictions
print('juntando span + ner, com pos processamento')
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)

def juntaPredictons(dic_predictions, dic_predictions_span):
    dic_predictions_all = {}
    for key, value in dic_predictions.items():
        #dic_predictions_all[key] = value.copy()
        tokens = value[0].copy()
        listaEntidadesNer = value[1].copy()
        listaIndicesNer=list()
        listaTipoNer=list()
        for entidadeNer in listaEntidadesNer:
            listaIndicesNer.append(entidadeNer[1])
            listaTipoNer.append(entidadeNer[2])
        #print('listaIndicesNer:', listaIndicesNer)
        listaEntidadesSpan = dic_predictions_span[key][1]
        for entidadeSpan in listaEntidadesSpan:
            indices = entidadeSpan[1]
            if indices in listaIndicesNer:
                #print('ja tem, pulando')
                pass
            else:
                #print('incluindo:', entidadeSpan)
                # só acrescenta se nao tem entidade mais externa do mesmo tipo
                # ver pelo indice
                isAninhada=False
                tipo=entidadeSpan[2]
                for indicesNer, tipoNer in zip(listaIndicesNer, listaTipoNer):
                    isAninhada = isEntidadeAninhada(indices, indicesNer) # é aninhada com alguma entidade?
                    if isAninhada and tipoNer == tipo:
                        break
                # ver pelo tipo tb
                if not isAninhada:
                    print('Não é aninhada, incluindo:', entidadeSpan)
                    listaEntidadesNer.append(entidadeSpan)
                else:
                    #print('aninhada:', entidadeSpan)
                    pass
        dic_predictions_all[key] = [tokens, listaEntidadesNer]
        #if key>15:
        #    break
    return dic_predictions_all

dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_span)
#dic_predictions_all
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

juntando span + ner, com pos processamento
Não é aninhada, incluindo: ['glicazida', [22], 'Problema']
Não é aninhada, incluindo: ['comprometimento difuso do VE', [33, 34, 35, 36], 'Anatomia']
Não é aninhada, incluindo: ['do VE', [35, 36], 'Anatomia']
Não é aninhada, incluindo: ['do VE grau', [35, 36, 37], 'Anatomia']
Não é aninhada, incluindo: ['VE', [36], 'Anatomia']
Não é aninhada, incluindo: ['VE grau', [36, 37], 'Anatomia']
Não é aninhada, incluindo: ['VE grau moderado', [36, 37, 38], 'Anatomia']
Não é aninhada, incluindo: ['grau', [37], 'Anatomia']
Não é aninhada, incluindo: ['insuf Mi', [40, 41], 'Teste']
Não é aninhada, incluindo: ['discreta', [42], 'Teste']
Não é aninhada, incluindo: ['insuf Tri', [44, 45], 'Anatomia']
Não é aninhada, incluindo: ['Tri discreta', [45, 46], 'Teste']
Não é aninhada, incluindo: ['discreta', [46], 'Teste']
Não é aninhada, incluindo: ['em MMII', [4, 5], 'Anatomia']
Não é aninhada, incluindo: ['MMII', [5], 'Anatomia']
Não é aninhada, incluindo: ['tora

In [183]:
print('agora, com  filtro pos tagger')

#dic_predictions_span = f.load_obj('dic_predictions_span_com_postagger_'+str(BATCH))
dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_span)
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

agora, com  filtro pos tagger
Não é aninhada, incluindo: ['glicazida', [22], 'Problema']
Não é aninhada, incluindo: ['VE', [36], 'Anatomia']
Não é aninhada, incluindo: ['VE grau', [36, 37], 'Anatomia']
Não é aninhada, incluindo: ['grau', [37], 'Anatomia']
Não é aninhada, incluindo: ['insuf Mi', [40, 41], 'Teste']
Não é aninhada, incluindo: ['insuf Tri', [44, 45], 'Anatomia']
Não é aninhada, incluindo: ['tróficas', [27], 'Teste']
Não é aninhada, incluindo: ['repolarização', [43], 'Teste']
Não é aninhada, incluindo: ['mmii', [4], 'Anatomia']
Não é aninhada, incluindo: ['folhetos', [2], 'Tratamento']
Não é aninhada, incluindo: ['CUSPIDES', [2], 'Tratamento']
Não é aninhada, incluindo: ['HB', [25], 'Anatomia']
Não é aninhada, incluindo: ['CICLOSPORINA', [5], 'Teste']
Não é aninhada, incluindo: ['VE', [0], 'Anatomia']
Não é aninhada, incluindo: ['VE', [7], 'Anatomia']
Não é aninhada, incluindo: ['GLOBAL', [15], 'Teste']
Não é aninhada, incluindo: ['GLOBAL MODERADAMENTE', [15, 16], 'Teste']


In [181]:
dic_predictions_span = f.load_obj('dic_predictions_span_com_postagger_'+str(BATCH))

dic_predictions_span[13]

[[['Ecocardiograma', 0],
  ['-', 1],
  ['ventrículo', 2],
  ['esquerdo', 3],
  ['com', 4],
  ['hipertrofia', 5],
  ['concentrica', 6],
  ['de', 7],
  ['grau', 8],
  ['discreto', 9],
  ['e', 10],
  ['função', 11],
  ['sistólica', 12],
  ['preservada', 13],
  ['.', 14]],
 [['ventrículo esquerdo', [2, 3], 'Anatomia'],
  ['hipertrofia concentrica de grau discreto', [5, 6, 7, 8, 9], 'Problema'],
  ['ventrículo', [2], 'Anatomia'],
  ['hipertrofia', [5], 'Problema'],
  ['hipertrofia concentrica', [5, 6], 'Problema']]]

In [179]:
dic_predictions_all[13]

[[['Ecocardiograma', 0],
  ['-', 1],
  ['ventrículo', 2],
  ['esquerdo', 3],
  ['com', 4],
  ['hipertrofia', 5],
  ['concentrica', 6],
  ['de', 7],
  ['grau', 8],
  ['discreto', 9],
  ['e', 10],
  ['função', 11],
  ['sistólica', 12],
  ['preservada', 13],
  ['.', 14]],
 [['Ecocardiograma', [0], 'Teste'],
  ['ventrículo esquerdo', [2, 3], 'Anatomia'],
  ['hipertrofia concentrica de grau discreto', [5, 6, 7, 8, 9], 'Problema']]]

In [184]:
dic_predictions_span

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['marevan 5mg', [12, 13], 'Tratamento'], ['marevan', [12], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-'

In [185]:
dic_predictions_all

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-', 34],
 